# Ноутбук для решения задач из 4 модуля

In [1]:
!pip install openai langchain langchain-openai langchain-community sentence_transformers faiss-cpu pypdf -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
from typing import List
from getpass import getpass

## Если используете ключ от OpenAI, запустите эту ячейку 👇







In [ ]:
from langchain_openai import ChatOpenAI
import os



# os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0)

## Если используете ключ из курса, запустите эти ячейки 👇


In [3]:
from langchain_openai import ChatOpenAI
from getpass import getpass

#course_api_key= "Введите ваш API ключ с курса"
course_api_key = getpass(prompt='Введите API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(api_key=course_api_key, model='gpt-4o-mini',
                 base_url="https://aleron-llm.neuraldeep.tech/")

Введите API ключ··········


## Задание 4.1.5 Кастомный Text Splitter

---



# *Напишите функцию*

In [4]:
from typing import List

def get_content_from_sections(text: str) -> List[str]:
    if not text.strip():
        return []

    lines = text.strip().split('\n')
    result = []
    current_content = []
    current_main_section = None

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Check if it's a main section (starts with single digit followed by dot and space)
        parts = line.split('.', 1)
        if (len(parts) == 2 and parts[0].isdigit() and len(parts[0]) == 1 and
            parts[1].startswith(' ')):

            # If we were collecting content for a previous main section, save it
            if current_main_section is not None and current_content:
                result.append(' '.join(current_content))
                current_content = []

            current_main_section = parts[0]

            # Check if there's content after the title on the same line
            title_and_content = parts[1].strip()
            if title_and_content:
                # Check if this is just a title or contains content
                # If the line contains only the title (no additional content), skip
                # We'll look for content in subsequent lines
                pass

        # Check if it's a subsection (like 1.1, 2.3, etc.) - ignore the title
        elif (len(line) > 3 and line[0].isdigit() and line[1] == '.' and
              line[2].isdigit() and (len(line) == 3 or line[3] == ' ' or line[3] == '.')):
            # This is a subsection title, we ignore it but will collect its content
            pass

        # It's a content line (not starting with digit-dot pattern)
        elif not (line[0].isdigit() and len(line) > 1 and line[1] == '.'):
            current_content.append(line)

    # Add the last section's content if it exists
    if current_main_section is not None and current_content:
        result.append(' '.join(current_content))

    return result

#Тексты из тестов


In [5]:

text1 = """
1. Вступление
Описание курса, начальные требования, для кого этот курс.
1.1 Как правильно входить в курс
Цели курса, канал и комьюнити, глоссарий, PET-проект, хард режим vs лайт режим?
1.2 Общий подход и точки улучшения приложений с LLM
Рассказываем почему важно разбираться в LLM. Нужен ли ИИ обычному человеку?
1.3 API ключ курса или от OpenAI?
1.3.1 Ключ от команды курса
Получаем ключ в боте.
1.3.2 Ключ от OpenAI
Получаем официальный ключ от OpenAI.
1.3.3 Ключ от HuggingFace
Ныряем в Open Source и получаем ключ от HuggingFace.
2. Промптинг - объясни LLM, что тебе от неё надо!
2.1 Введение в Prompt Engineering
Поясняем за промпты. Техники и лайфхаки для промптинга. Из чего состоит промпт?
2.2 Дизайн промптов в LangChain
Few-shot learning. Output Parser.
2.2.1 Введение в LangChain
Рассказываем про преимущества LangChain.
3. LangChain или причем тут попугаи?
3.1 Память в LangChain
Переводим LLM в чат-режим. Типы памяти.
3.2 Chains - собери свою цепь
Chains & LCEL.
3.3 Агенты intro
Агенты и цепи. Инструменты (tools).
"""

text2 = """


"""

text3 = """
1. Главный заголовок
Тут содержимое, которое нужно достать из главного заголовка.
1.1 Подзаголовок
Тут содержимое, которое нужно достать из подзаголовка.
"""

text4 = """
1. Первый заголовок

2. Второй заголовок
"""

text5 = """
1. Введение
Краткое описание целей и задач документа.

2. Основные разделы
Краткий обзор основных разделов.

2.1 Первый подраздел
Какие-то мелкие детали.

3. Заключение
Выводы и заключительные комментарии.
4. Приложения
Дополнительная информация и материалы.
"""

# Задание 4.11.1 📚 Литературный RAG из pdf 🤖

Загружаем файлы

In [6]:
!wget https://stepik.org/media/attachments/lesson/1084288/The_Daughter_of_The_Commandant.pdf

--2025-09-26 12:03:24--  https://stepik.org/media/attachments/lesson/1084288/The_Daughter_of_The_Commandant.pdf
Resolving stepik.org (stepik.org)... 178.248.239.111
Connecting to stepik.org (stepik.org)|178.248.239.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198595 (194K) [application/pdf]
Saving to: ‘The_Daughter_of_The_Commandant.pdf’

The_Daughter_of_The 100%[===================>] 193.94K   452KB/s    in 0.4s    

2025-09-26 12:03:25 (452 KB/s) - ‘The_Daughter_of_The_Commandant.pdf’ saved [198595/198595]



In [7]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('https://stepik.org/media/attachments/lesson/1084288/pushkin_questions.csv')

Соберем все компоненты для RAG


<center> <img src='https://github.com/a-milenkin/LLM_practical_course/blob/main/images/RAG.png?raw=1' width="800" height="250">


In [8]:
# Определите Document loader для pdf
import pandas as pd
from langchain.document_loaders import CSVLoader, DataFrameLoader, PyPDFLoader

# Загрузка PDF файла
loader = PyPDFLoader("/content/The_Daughter_of_The_Commandant.pdf")
documents = loader.load()

Посмотрите, что хранится в переменной documents - может быть сплитер вам не нужен.

In [24]:
# Определите подходящий Splitter
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]  # Приоритет разделителей
)
split_documents = splitter.split_documents(documents)

print(f"Создано {len(split_documents)} чанков")

# Проверка результата
print("Первый чанк:")
print(split_documents[0].page_content[:200] + "...")

Создано 29 чанков
Первый чанк:
www.a4format.ru  
Все произведения школьной программы в кратком изложении. — М.: АСТ, 1996.  
 
И.О. Родин, Т.М. Пименова 
Капитанская дочка 
Краткое содержание 
Глава 1 
Сержант гвардии 
Глава открыв...


# Выбираем Embedding models

## 🔱 Эмбеддинги от OpenAI (API)

In [26]:
# # Если используете ключ от OpenAI запустите эту ячейку
# from langchain_openai import OpenAIEmbeddings

# embeddings_api_model = OpenAIEmbeddings()

In [27]:
# Если используете ключ курса, запустите эту ячейку
from langchain_openai import OpenAIEmbeddings

embeddings_api_model = OpenAIEmbeddings(api_key=course_api_key, model='text-embedding-3-small',
                                        base_url="https://aleron-llm.neuraldeep.tech/")

## 🔱 Эмбеддинги от HuggingFace 🤗
Переключитесь на среду с GPU, если хотите использовать видеокарту.

In [28]:
from langchain.embeddings import HuggingFaceEmbeddings

# Если у вас нет видеокарты, укажите 'device': 'cpu'
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru", model_kwargs={"device": "cpu"}
)

# Выбираем 🗂 Vector Store

In [29]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

In [30]:
db = FAISS.from_documents(
    split_documents, hf_embeddings_model
)

db.save_local("/content/faiss_db")  # можно сохранить базу локально, указав путь

In [31]:
# База готова, теперь можно делать к ней запросы
db.similarity_search("Что там за дочка?")

[Document(id='51c54dec-7fd4-4e13-adc2-cf588c579e73', metadata={'producer': 'Acrobat Distiller 9.0.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2009-12-24T14:33:47+03:00', 'author': '<CCF3F1E0F2EEE2E0>', 'moddate': '2009-12-24T14:33:47+03:00', 'title': '<4D6963726F736F667420576F7264202D20CAC45FE8E7EB5FD0EEE4E8ED>', 'source': '/content/The_Daughter_of_The_Commandant.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='Швабриным, молодым офицером « невысокого роста, с лицом смуглым и отменно не-\nкрасивым, но чрезвычайно живым». Швабрин переведен в крепость за дуэль. Швабрин \nрассказывает Гриневу о жизни в крепости, описывает семью коменданта, особенно'),
 Document(id='d5c5abda-5492-4dfd-97f9-5d5bcc3ca308', metadata={'producer': 'Acrobat Distiller 9.0.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2009-12-24T14:33:47+03:00', 'author': '<CCF3F1E0F2EEE2E0>', 'moddate': '2009-12-24T14:33:47+03:00', 'title': '<4D6963726F736F

# Определим 🎣 Retriever

In [34]:
# Самый частый кейс - использование векторного хранилища и его методов для получения документов
retriever = db.as_retriever(
  #  search_type="similarity",  # тип поиска похожих документов
    k=4,  # количество возвращаемых документов (Default: 4)
    score_threshold=None,  # минимальный порог для поиска "similarity_score_threshold"
)
retriever.get_relevant_documents(
    "Кто такая Маша?"
)

[Document(id='0fb4c28a-3d90-424d-9e21-3a5c428c137b', metadata={'producer': 'Acrobat Distiller 9.0.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2009-12-24T14:33:47+03:00', 'author': '<CCF3F1E0F2EEE2E0>', 'moddate': '2009-12-24T14:33:47+03:00', 'title': '<4D6963726F736F667420576F7264202D20CAC45FE8E7EB5FD0EEE4E8ED>', 'source': '/content/The_Daughter_of_The_Commandant.pdf', 'total_pages': 4, 'page': 1, 'page_label': '2'}, page_content='Гринев много общается со Швабриным, но тот ему нравится все меньше, а особенно его \nколкие замечания о Маше. Гринев посвящает Маше посредственные любовные стихи. \nШвабрин резко их критикует, в разговоре с Гриневым оскорбляет Машу. Гринев называет \nего лжецом, Швабрин требует сатисфакции. Перед дуэлью по приказанию Василисы \nЕгоровны их арестовывают, дворовая девка Палашка даже отнимает у них шпаги. Через \nнекоторое время Гринев узнает от Маши, что Швабрин за нее сватался, а она отказала (этим'),
 Document(id='b9e5a235-19d0-4c0

# 🚰 RAG Pipeline - подключаем RAG к LLM

In [36]:
from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Создаём простой шаблон
template = """
Answer the question based only on the following context:

{context}

Question: {question}
"""
# Создаём промпт из шаблона
prompt = ChatPromptTemplate.from_template(template)


# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


# Создаём цепочку
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke("Кто такая Маша?")


'Маша — это девушка, к которой испытывает чувства Гринев. Она является объектом внимания Швабрина, который сватался к ней, но была вынуждена отказать ему. Маша заботится о Гринева, когда он ранен, и в конечном итоге Гринев делает ей предложение. Также она просит императрицу о милости для Гринева.'

In [37]:
chain.invoke("Кто такая капитанская дочка?")

'Капитанская дочка — это Маша, дочь капитана Миронова. Она является одной из ключевых персонажей в произведении "Капитанская дочка" Александра Пушкина.'

In [38]:
chain.invoke("В чем поучительный смысл произведения?")

'Поучительный смысл произведения "Капитанская дочка" заключается в том, что оно акцентирует внимание на важности чести, мужества и верности в трудных жизненных ситуациях. Герой, Петр Гринев, учится принимать ответственные решения и отстаивать свои моральные ценности, даже когда обстоятельства ставят его перед сложным выбором. Также произведение подчеркивает значимость любви, дружбы и преданности, которые способны преодолеть любые трудности.'

In [43]:
from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Создаём простой шаблон
template = """
Answer the question based only on the following context. Сделай ответ до 70 символов. И давай ответ по факту, коротко и по делу:

{context}

Question: {question}
"""
# Создаём промпт из шаблона
prompt = ChatPromptTemplate.from_template(template)


# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


# Создаём цепочку
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [44]:
answers = [] # Список, где будем хранить ответы модели

for text_input in tqdm(df['question']):
    answer = chain.invoke(text_input)
    answers.append(answer) # Добавляем ответ в список
  #  break # Для отладки. Уберите, когда убедитесь, что на одном примере работает

100%|██████████| 15/15 [00:19<00:00,  1.27s/it]


In [45]:
answers

['Пугачева убили раздетой на крыльце.',
 'В Царском Селе.',
 'Гринев и Савельич сбились с дороги из-за бурана.',
 'Швабрин издевался над Машей и морил ее голодом.',
 'В Сибирь.',
 'В семье Петра Гринева было девять детей.',
 'Петра Гринева воспитывал дядька Савельич.',
 'Гринев попросил полсотни казаков.',
 'Белогорская крепость находилась в 40 верст от города.',
 'Дочку коменданта звали Маша.',
 'Петя подарил вожатому платок.',
 'Береги платье снову, а честь смолоду.',
 'Пугачев оставил комендантом Швабрина.',
 'Швабрина перевели в крепость за дуэль.',
 '«Барского дитятю» (младшего сына помещика).']

In [46]:
df['answer'] = answers # Создаём новый столбец из ответов модели

In [47]:
df.to_csv('4_1_11_solution.csv', index=False) # Сохраняем файл, отправляем на Stepik, получаем баллы :)